### Проекционные методы решения краевой задачи для обыкновенного дифференциального уравнения второго порядка

Вариант 1

Дана граничная задача

$-(\frac{1}{2+x}u')' + cos(x)u = 1+x$

$u(-1) = u(1) = 0$

#### Метод Ритца

Координатная система: $(1-x^2)P_i^{(1,1)}(x), \ i=0, 1, ...$

In [15]:
import basis_functions as bf
import numpy as np
import sympy
import pandas as pd

# возвращает набор коэффициентов c_i разложения U
# с использованием данных функций f, p, r
def Ritz_method(k, n):
    pols = bf.Jacobi(k, n)
    phis, dphis = bf.base_funs(k, n)

    A = np.zeros((n, n))
    b = np.zeros((n, 1))

    x = sympy.symbols('x')
    f = 1 + x
    for i in range(3):
        h = f * phis[i]
        b[i] = sympy.integrals.integrate(h, (x, -1, 1))

    # вспомогательные значения для интегрирования Гаусса
    x1 = 1 / 3 * np.sqrt(5 - 2 * np.sqrt(10 / 7))
    x2 = 1 / 3 * np.sqrt(5 + 2 * np.sqrt(10 / 7))
    c1 = (322 + 13 * np.sqrt(70)) / 900
    c2 = (322 - 13 * np.sqrt(70)) / 900
    x_i = [-x2, -x1, 0, x1, x2]
    c_i = [c2, c1, 128 / 225, c1, c2]

    # первый индекс - номер узла
    # второй - определеяет производная или просто значение (0 - значение, 1 - производная)
    # третий - номер функци Ф_i
    arr = [bf.base_funs_values(k, n, x_i[i]) for i in range(5)]

    def Gauss(nodes, coefs, i, j):
        s = 0
        for k in range(len(nodes)):
            tmp_1 = ((1 / (2 + nodes[k])) * arr[k][1][j] * arr[k][1][i] + np.cos(nodes[k]) * arr[k][0][i] * arr[k][0][j])
            s += coefs[k] * tmp_1
        return s


    for i in range(n):
        for j in range(n):
            A[i][j] = Gauss(x_i, c_i, i, j)

    coeffs = np.linalg.solve(A, b)
    return coeffs


#### Метод коллокации
Узлы $t_i$ - узлы Чебышева

Координатная система: $(1-x^2)P_i^{(1,1)}(x), \ i=0, 1, ...$

In [16]:
def Chebyshev_nodes(n, a, b):
    arr = []
    for i in range(1, n + 1):
        tmp1 = (1 / 2) * (a + b)
        tmp2 = (1 / 2) * (b - a)
        #arr[i] = tmp1 + tmp2 * np.cos((2 * i - 1) * np.pi /( 2 * n))
        arr.append(np.cos((2 * i - 1) * np.pi / (2 * n)))
    return arr

def collocation_method(k, n):
    nodes = Chebyshev_nodes(n, -1, 1)
    f = lambda x: 1 + x
    p = lambda x: 1 / (2 + x)
    dp = lambda x: (-1) / (2 + x) ** 2
    r = lambda x: np.cos(x)

    A = np.zeros((n, n))
    b = np.zeros((n, 1))

    for i in range(n):
        b[i] = f(nodes[i])
        phi, dphi = bf.base_funs_values(k, n, nodes[i])
        ddphi = bf.base_funs_val_ddphi(k, n, nodes[i])
        for j in range(n):
            tmp1 = p(nodes[i]) * ddphi[j]
            tmp2 = dp(nodes[i]) * dphi[j]
            tmp3 = r(nodes[i]) * phi[j]
            A[i][j] = (-1) * (tmp1 + tmp2) + tmp3
    coeffs = np.linalg.solve(A, b)
    return coeffs


Вычисление значения полученного решения при данных коэффициентах

In [17]:
def final_solution(coes, dots):
    dot1, dot2, dot3 = dots[0], dots[1], dots[2]

    # значение полученное методом Рунге-Кутта с шагом 0.1
    exact_value = [0.270198, 0.529251, 0.632899]
    res = [0.0] * 3
    n = len(coes)

    phi_dot1 = bf.base_funs_values(1, n, dot1)[0]
    phi_dot2 = bf.base_funs_values(1, n, dot2)[0]
    phi_dot3 = bf.base_funs_values(1, n, dot3)[0]

    for i in range(3):
        res[0] += coes[i] * phi_dot1[i]
        res[1] += coes[i] * phi_dot2[i]
        res[2] += coes[i] * phi_dot3[i]

    errs = [exact_value[k] - res[k] for k in range(3)]
    arr = [np.round(res[0], 5),
           np.round(res[1], 5),
           np.round(res[2], 5),
           np.round(errs[0], 5),
           np.round(errs[1], 5),
           np.round(errs[2], 5)]
    return arr


In [18]:
# печать таблицы результатов
def make_table(values):
    column = [
        "y(-0.5)",
        "y(0)",
        "y(0.5)",
        "y* - y _(-0.5)",
        "y* - y _(0.0)",
        "y* - y _(0.5)"
    ]
    indexes = [3, 4, 5, 6, 7]
    table = pd.DataFrame(data = values, columns=column, index=indexes)
    table.columns.name = "n"
    return(table)

Вычисляем решение методом Ритца

In [19]:
dots = [-0.5, 0.0, 0.5]
value_Ritz = []
for i in range(3, 8):
    coeffs = Ritz_method(1, i)
    value_Ritz.append(final_solution(coeffs, dots))
result_table = make_table(value_Ritz)
result_table

n,y(-0.5),y(0),y(0.5),y* - y _(-0.5),y* - y _(0.0),y* - y _(0.5)
3,[0.26742],[0.54346],[0.61822],[0.00278],[-0.0142],[0.01468]
4,[0.26784],[0.54148],[0.61948],[0.00236],[-0.01223],[0.01342]
5,[0.26799],[0.54219],[0.61933],[0.00221],[-0.01294],[0.01357]
6,[0.26066],[0.54475],[0.63201],[0.00954],[-0.0155],[0.00089]
7,[0.37366],[0.25301],[0.75022],[-0.10346],[0.27624],[-0.11733]


Вычисляем решение методом коллокации

In [20]:
dots = [-0.5, 0.0, 0.5]
value_collocate = []
for i in range(3, 8):
    coeffs = collocation_method(1, i)
    value_collocate.append(final_solution(coeffs, dots))
result_table = make_table(value_collocate)
result_table


n,y(-0.5),y(0),y(0.5),y* - y _(-0.5),y* - y _(0.0),y* - y _(0.5)
3,[0.23852],[0.53876],[0.65014],[0.03168],[-0.00951],[-0.01725]
4,[0.26874],[0.53824],[0.61699],[0.00146],[-0.00898],[0.01591]
5,[0.26765],[0.54161],[0.61988],[0.00254],[-0.01236],[0.01302]
6,[0.26785],[0.54126],[0.61968],[0.00235],[-0.01201],[0.01322]
7,[0.26782],[0.54128],[0.61962],[0.00238],[-0.01203],[0.01328]
